<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Text-Classification" data-toc-modified-id="Text-Classification-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Text Classification</a></span><ul class="toc-item"><li><span><a href="#Subjectivity-Dataset" data-toc-modified-id="Subjectivity-Dataset-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Subjectivity Dataset</a></span></li><li><span><a href="#Tokenization" data-toc-modified-id="Tokenization-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Tokenization</a></span><ul class="toc-item"><li><span><a href="#Simple-Tokenization" data-toc-modified-id="Simple-Tokenization-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Simple Tokenization</a></span></li><li><span><a href="#Much-better-tokenization-with-Spacy" data-toc-modified-id="Much-better-tokenization-with-Spacy-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Much better tokenization with Spacy</a></span></li></ul></li><li><span><a href="#Split-dataset-in-train-and-test" data-toc-modified-id="Split-dataset-in-train-and-test-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Split dataset in train and test</a></span></li><li><span><a href="#Word-to-index-mapping" data-toc-modified-id="Word-to-index-mapping-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Word to index mapping</a></span></li><li><span><a href="#Sentence-encoding" data-toc-modified-id="Sentence-encoding-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Sentence encoding</a></span></li><li><span><a href="#Embedding-layer" data-toc-modified-id="Embedding-layer-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Embedding layer</a></span></li><li><span><a href="#Continuous-Bag-of-Words-Model" data-toc-modified-id="Continuous-Bag-of-Words-Model-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Continuous Bag of Words Model</a></span></li></ul></li><li><span><a href="#Training-the-CBOW-model" data-toc-modified-id="Training-the-CBOW-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training the CBOW model</a></span></li><li><span><a href="#Data-loaders-for-SGD" data-toc-modified-id="Data-loaders-for-SGD-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data loaders for SGD</a></span></li></ul></div>

In [1]:
# import pytorch libraries
%matplotlib inline
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# Text Classification
In this part of the tutorial we develop a continuous bag of words (CBOW) model for a text classification task described [here]( https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf). The CBOW model was first described [here](https://arxiv.org/pdf/1301.3781.pdf)

## Subjectivity Dataset
The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [4]:
def unpack_dataset():
    ! wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
    ! mkdir data
    ! tar -xvf rotten_imdb.tar.gz -C data

In [5]:
#unpack_dataset()

--2019-02-20 15:16:44--  http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.20
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519599 (507K) [application/x-gzip]
Saving to: ‘rotten_imdb.tar.gz’

rotten_imdb.tar.gz  100%[===================>] 507.42K  --.-KB/s    in 0.01s   

2019-02-20 15:16:45 (47.5 MB/s) - ‘rotten_imdb.tar.gz’ saved [519599/519599]

x quote.tok.gt9.5000
x plot.tok.gt9.5000
x subjdata.README.1.0


In [6]:
!ls data

plot.tok.gt9.5000   quote.tok.gt9.5000  subjdata.README.1.0


In [7]:
! head -2 data/plot.tok.gt9.5000

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism , minimalism and russian constructivism , graffiti removal has secured its place in the history of modern art while being created by artists who are unconscious of their artistic achievements . 


In [8]:
from pathlib import Path
PATH = Path("data")
list(PATH.iterdir())

[PosixPath('data/plot.tok.gt9.5000'),
 PosixPath('data/subjdata.README.1.0'),
 PosixPath('data/quote.tok.gt9.5000')]

## Tokenization
Tokenization is the task of chopping up text into pieces, called tokens.

spaCy is an open-source software library for advanced Natural Language Processing. Here we will use it for tokenization.  

### Simple Tokenization

In [9]:
# We need each line in the file 
def read_file(path):
    """ Read file returns a list of lines.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = f.readlines()
    return content

In [10]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [11]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [12]:
np.array(obj_lines[0].strip().lower().split(" "))

array(['the', 'movie', 'begins', 'in', 'the', 'past', 'where', 'a',
       'young', 'boy', 'named', 'sam', 'attempts', 'to', 'save', 'celebi',
       'from', 'a', 'hunter', '.'], dtype='<U8')

### Much better tokenization with Spacy

In [ ]:
#!pip install -U spacy

In [ ]:
import spacy

In [ ]:
# first time run this
#!python3 -m spacy download en

In [ ]:
tok = spacy.load('en')

In [ ]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [ ]:
len(obj_lines)

In [ ]:
obj_lines[0]

In [ ]:
test = tok(obj_lines[0])

In [ ]:
np.array([x for x in test])

## Split dataset in train and test

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([line.strip().lower() for line in sub_content])
obj_content = np.array([line.strip().lower() for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [15]:
X[0], y[0]

('smart and alert , thirteen conversations about one thing is a small gem .',
 0.0)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
X_train[:5], y_train[:5]

(array(['will god let her fall or give her a new path ?',
        "the director's twitchy sketchbook style and adroit perspective shifts grow wearisome amid leaden pacing and indifferent craftsmanship ( most notably wretched sound design ) .",
        "welles groupie/scholar peter bogdanovich took a long time to do it , but he's finally provided his own broadside at publishing giant william randolph hearst .",
        'based on the 1997 john king novel of the same name with a rather odd synopsis : " a first novel about a seasoned chelsea football club hooligan who represents a disaffected society operating by brutal rules .',
        'yet , beneath an upbeat appearance , she is struggling desperately with the emotional and physical scars left by the attack .'],
       dtype='<U691'), array([1., 0., 0., 1., 1.]))

## Word to index mapping
In interest of time we will tokenize without spaCy. Here we will compute a vocabulary of words based on the training set and a mapping from word to an index.

In [18]:
from collections import defaultdict

In [19]:
def get_vocab(content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for line in content:
        words = set(line.split())
        for word in words:
            vocab[word] += 1
    return vocab      

In [36]:
#Getting the vocabulary from the training set
word_count = get_vocab(X_train)

In [43]:
#word_count

In [22]:
len(word_count.keys())

21415

In [23]:
# let's delete words that are very infrequent
for word in list(word_count):
    if word_count[word] < 5:
        del word_count[word]

In [24]:
len(word_count.keys())

4065

In [25]:
## Finally we need an index for each word in the vocab
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [45]:
#vocab2index

## Sentence encoding
Here we encode each sentence as a sequence of indices corresponding to each word.

In [41]:
x_train_len = np.array([len(x.split()) for x in X_train])
x_test_len = np.array([len(x.split()) for x in X_test])

In [28]:
np.percentile(x_train_len, 95) # let set the max sequence len to N=40

43.0

In [29]:
X_train[0]

'will god let her fall or give her a new path ?'

In [30]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("?", vocab2index["UNK"])

7

In [31]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

array([ 8, 12, 11,  5,  3,  4,  9,  5, 10,  6,  2,  7])

In [32]:
def encode_sentence(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc

In [33]:
encode_sentence(X_train[0])

array([ 8, 12, 11,  5,  3,  4,  9,  5, 10,  6,  2,  7,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int32)

In [34]:
x_train = np.vstack([encode_sentence(x) for x in X_train])
x_train.shape

(8000, 40)

In [35]:
x_test = np.vstack([encode_sentence(x) for x in X_test])
x_test.shape

(2000, 40)

## Embedding layer
Most deep learning models use a dense vectors of real numbers as representation of words (word embeddings), as opposed to a one-hot encoding representations. The module torch.nn.Embedding is used to represent word embeddings. It takes two arguments: the vocabulary size, and the dimensionality of the embeddings. The embeddings are initialized with random vectors. 

In [67]:
# an Embedding module containing 10 words with embedding size 4
# embedding will be initialized at random
embed = nn.Embedding(10, 4, padding_idx=0)
embed.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.8536,  0.1352, -0.2728,  0.4877],
        [ 0.1901,  1.3434,  2.4007, -1.3473],
        [ 1.9391,  0.4154,  1.8454, -0.0440],
        [-0.8543,  1.9308, -0.4309, -0.5002],
        [-1.2015,  0.6970, -0.4598,  0.1874],
        [-0.5489,  1.1735, -1.6129, -0.3163],
        [ 1.2211, -0.1239,  0.3409, -1.1177],
        [ 0.5130, -1.4622,  0.4938,  0.4620],
        [-0.0666,  1.7076, -2.7429, -0.1095]])

Note that the `padding_idx` has embedding vector 0.

In [68]:
# given a list of ids we can "look up" the embedding corresponing to each id
# can you see that some vectors are the same?
a = torch.LongTensor([[1,4,1,5,1,0]])
embed(a)

tensor([[[ 0.8536,  0.1352, -0.2728,  0.4877],
         [-0.8543,  1.9308, -0.4309, -0.5002],
         [ 0.8536,  0.1352, -0.2728,  0.4877],
         [-1.2015,  0.6970, -0.4598,  0.1874],
         [ 0.8536,  0.1352, -0.2728,  0.4877],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]])

This would be the representation of a sentence with words with indices [1,4,1,5,1] and a padding at the end. Bellow we have an example in which we have two sentences. the first sentence has length 3 and the last sentence has length 2. In order to use a tensor we use padding at the end of the second sentence. 

In [69]:
a = torch.LongTensor([[1,4,1], [1,3,0]])

Our model takes an average of the word embedding of each word. Here is how we do it.

In [70]:
s = torch.FloatTensor([3, 2]) # here is the size of the vector

In [71]:
embed(a)

tensor([[[ 0.8536,  0.1352, -0.2728,  0.4877],
         [-0.8543,  1.9308, -0.4309, -0.5002],
         [ 0.8536,  0.1352, -0.2728,  0.4877]],

        [[ 0.8536,  0.1352, -0.2728,  0.4877],
         [ 1.9391,  0.4154,  1.8454, -0.0440],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]])

In [72]:
embed(a).sum(dim=1)

tensor([[ 0.8529,  2.2012, -0.9765,  0.4752],
        [ 2.7927,  0.5506,  1.5726,  0.4437]])

In [73]:
sum_embs = embed(a).sum(dim=1) 
sum_embs/ s.view(s.shape[0], 1)

tensor([[ 0.2843,  0.7337, -0.3255,  0.1584],
        [ 1.3964,  0.2753,  0.7863,  0.2219]])

## Continuous Bag of Words Model

In [74]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, emb_size=100):
        super(CBOW, self).__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.linear = nn.Linear(emb_size, 1)
        
    def forward(self, x, s):
        x = self.word_emb(x)
        x = x.sum(dim=1)/ s.view(s.shape[0], 1)
        x = self.linear(x)
        return x

In [75]:
model = CBOW(vocab_size=5, emb_size=3)

In [76]:
model.word_emb.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.6823, -1.3827, -0.1621],
        [-0.0323, -0.2556,  2.5525],
        [-1.2646, -0.0038,  1.4429],
        [-0.5084, -0.4949, -0.3695]])

In [77]:
model(a, s)

tensor([[ 0.5973],
        [ 0.7232]])

# Training the CBOW model 

In [78]:
V = len(words)
model = CBOW(vocab_size=V, emb_size=50)
print(V)

4067


In [79]:
def test_metrics(model):
    model.eval()
    x = torch.LongTensor(x_test) #.cuda()
    y = torch.Tensor(y_test).unsqueeze(1) #).cuda()
    s = torch.Tensor(x_test_len).view(x_test_len.shape[0], 1)
    y_hat = model(x, s)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct = (y_pred.float() == y).float().sum()
    accuracy = correct/y_pred.shape[0]
    print("test loss %.3f and accuracy %.3f" % (loss.item(), accuracy.item()))

In [80]:
# accuracy of a random model should be around 0.5
test_metrics(model)

test loss 0.702 and accuracy 0.477


In [81]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    for i in range(epochs):
        x = torch.LongTensor(x_train)  #.cuda()
        y = torch.Tensor(y_train).unsqueeze(1)
        s = torch.Tensor(x_train_len).view(x_train_len.shape[0], 1)
        y_hat = model(x, s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss.item())
    test_metrics(model)

In [82]:
train_epocs(model, epochs=10, lr=0.1)

0.7024547457695007
0.6619945168495178
0.5833789110183716
0.4843262732028961
0.37453681230545044
0.2894887924194336
0.2270020842552185
0.1904212087392807
0.16081099212169647
0.14118488132953644
test loss 0.264 and accuracy 0.902


In [83]:
train_epocs(model, epochs=10, lr=0.01)

0.12197427451610565
0.11815891414880753
0.11501367390155792
0.1116267591714859
0.10820414870977402
0.10499171167612076
0.10200171917676926
0.09909720718860626
0.09619797766208649
0.09334372729063034
test loss 0.263 and accuracy 0.908


In [ ]:
train_epocs(model, epochs=10, lr=0.01)

# Data loaders for SGD

Nearly all of deep learning is powered by one very important algorithm: **stochastic gradient descent (SGD)**. SGD can be seeing as an approximation of **gradient descent** (GD). In GD you have to run through *all* the samples in your training set to do a single itaration. In SGD you use *only one* or *a subset*  of training samples to do the update for a parameter in a particular iteration. The subset use in every iteration is called a **batch** or **minibatch**.

In [84]:
from torch.utils.data import Dataset, DataLoader

Next we are going to create a data loader. The data loader provides the following features:
* Batching the data
* Shuffling the data
* Load the data in parallel using multiprocessing workers.

In [101]:
def encode_sentence2(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc, l

In [102]:
encode_sentence2(X_train[0])

(array([12,  6,  9,  2, 10,  4,  3,  2,  8,  5, 11,  7,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0], dtype=int32), 12)

In [103]:
class SubjectivityDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        x, s = encode_sentence2(x)
        return x, self.y[idx], s
    
sub_dataset_train = SubjectivityDataset(X_train, y_train)

In [104]:
train_loader = DataLoader(sub_dataset_train, batch_size=5, shuffle=True)
x, y, s = next(iter(train_loader))

In [106]:
x, y, s

(tensor([[  118,    16,  1273,  1359,    60,  3632,    60,    97,    28,
            931,    64,     8,  1258,    30,    37,   497,  1035,     1,
             18,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [    1,     1,     1,    17,     1,   475,   869,    60,     1,
              1,    30,    60,    59,    82,    16,     1,    51,     1,
              1,    28,   137,  2143,  2285,    18,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [    8,   128,   758,  1742,     1,  1834,     1,  3657,    17,
              1,    18,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
       

In [128]:
model = CBOW(vocab_size=V, emb_size=50)

In [129]:
train_loader = DataLoader(sub_dataset_train, batch_size=500, shuffle=True)

In [130]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    for i in range(epochs):
        for x, y, s in train_loader:
            x = x.type(torch.LongTensor)  #.cuda()
            y = y.type(torch.FloatTensor).unsqueeze(1)
            s = s.type(torch.Tensor).view(s.shape[0], 1)
            y_hat = model(x, s)
            loss = F.binary_cross_entropy_with_logits(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(loss.item())
    test_metrics(model)

In [131]:
train_epocs(model, epochs=20)

0.595077395439148
0.4045970141887665
0.26646387577056885
0.19933322072029114
0.16005471348762512
0.140223428606987
0.12022583186626434
0.11802416294813156
0.08472650498151779
0.08180411905050278
0.08018757402896881
0.08355165272951126
0.06486327946186066
0.05094480887055397
0.051483701914548874
0.04123803228139877
0.03242563083767891
0.02988939918577671
0.028660159558057785
0.025195850059390068
test loss 0.336 and accuracy 0.890
